In [3]:
import requests
import pandas as pd
import getpass

#import specific wrapper libs
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [4]:
password = getpass.getpass()

In [5]:
sp = spotipy.Spotify(auth_manager= SpotifyClientCredentials(client_id= '23d0a444d9634a749183dedd869e2ed0'  , client_secret= password))
sp

In [ ]:
results = sp.user_playlist_tracks(playlist_id='6FKDzNYZ8IW1pvYVF4zUN2')
results

{'href': 'https://api.spotify.com/v1/playlists/6FKDzNYZ8IW1pvYVF4zUN2/tracks?offset=0&limit=100&additional_types=track',
 'items': [{'added_at': '2017-08-03T23:41:30Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/exiedous'},
    'href': 'https://api.spotify.com/v1/users/exiedous',
    'id': 'exiedous',
    'type': 'user',
    'uri': 'spotify:user:exiedous'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0TnOYISbd1XYRBk9myaseg'},
       'href': 'https://api.spotify.com/v1/artists/0TnOYISbd1XYRBk9myaseg',
       'id': '0TnOYISbd1XYRBk9myaseg',
       'name': 'Pitbull',
       'type': 'artist',
       'uri': 'spotify:artist:0TnOYISbd1XYRBk9myaseg'}],
     'available_markets': ['AR',
      'BO',
      'CA',
      'CL',
      'CO',
      'CR',
      'DO',
      'EC',
      'SV',
      'GT',
      'HN',
      'HK',
      'IE',
      '

In [6]:
def playlists_data(playlist_id):
    df = []

    for count in range(0, 10000, 100):
        results = sp.user_playlist_tracks(user='spotify', playlist_id=playlist_id, limit=100, offset=count)

        tracks = results['items']
        for item in tracks:
            artist_names = [artist['name'] for artist in item['track']['artists']]
            artists_string = ", ".join(artist_names)
            df.append({
                    'Track_id': item['track']['id'],
                    'Title': item['track']['name'],
                    'Artist_id': item['track']['artists'][0]['id'],
                    'Artists': artists_string,
                    'Album': item['track']['album']['name'],
                    'Album Release Date': item['track']['album']['release_date']
                })

    return pd.DataFrame(df)



In [7]:
df1 = playlists_data('6FKDzNYZ8IW1pvYVF4zUN2')
df1

,Track_id,Title,Artist_id,Artists,Album,Album Release Date
0,3tgZ9vmhuAY9wEoNUJskzV,Timber (feat. Ke$ha),0TnOYISbd1XYRBk9myaseg,"Pitbull, Kesha",Global Warming: Meltdown (Deluxe Version),2012
1,2EBCVPNAG46nbgs6jXPGvv,Starships,0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,Pink Friday: Roman Reloaded The Re-Up (Explici...,2012-01-01
2,6pGUGTIaZ1H4jKHIL4Fged,Wild Strawberries,6n28c9qs9hNGriNa72b26u,PNAU,PNAU (Tour Edition),2009-01-01
3,09TcIuH1ZO7i4vicWKoaN2,Papaoutai,5j4HeCoUlzhfWtjAfM1acR,Stromae,racine carrée,2013-01-01
4,1TfqLAPs4K3s2rJMoCokcS,Sweet Dreams (Are Made of This) - Remastered,0NKDgy9j66h3DLnN8qu1bB,"Eurythmics, Annie Lennox, Dave Stewart",Sweet Dreams (Are Made Of This),1983-01-04
...,...,...,...,...,...,...
9995,6VfNTf0N1HwfFKl7Y18diU,Omen,4k1ELeJKT1ISyDv8JivPpB,The Prodigy,Invaders Must Die,2009-02-23
9996,28d1X9lfagOD4iFULH4qEK,Dark Star - Homemade Weapons Remix,2rekrvrzMex0PPBX6zvvfj,"Quadrant, Iris, Homemade Weapons",Form Constant EP,2020-04-24
9997,5HiSc2ZCGn8L3cH3qSwzBT,러시안 룰렛 Russian Roulette,1z4g3DjTBBZKhvAroFlhOM,Red Velvet,Russian Roulette - The 3rd Mini Album,2016-09-07
9998,2rgtCOVK2tpfao2VjURn84,Lovers on the Sun (feat. Sam Martin),1Cs0zKBU1kc0i8ypK3B9ai,"David Guetta, Sam Martin",Listen,2014-11-10


In [8]:
def audio_data(df,column):
    track_ids = df[column].tolist()
    chunk_size=50
    chunks = [track_ids[i:i + chunk_size] for i in range(0, len(track_ids), chunk_size)]
    data = []

    for chunk in chunks:
        features = sp.audio_features(chunk)
        data.extend(features)

    return pd.DataFrame(data)

In [9]:
df2 = audio_data(df1, 'Track_id')
df2

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.589,0.966,11,-4.106,1,0.0972,0.030000,0.000000,0.1470,0.8020,129.983,audio_features,3tgZ9vmhuAY9wEoNUJskzV,spotify:track:3tgZ9vmhuAY9wEoNUJskzV,https://api.spotify.com/v1/tracks/3tgZ9vmhuAY9...,https://api.spotify.com/v1/audio-analysis/3tgZ...,204200,4
1,0.747,0.716,11,-2.457,0,0.0750,0.135000,0.000000,0.2510,0.7510,125.008,audio_features,2EBCVPNAG46nbgs6jXPGvv,spotify:track:2EBCVPNAG46nbgs6jXPGvv,https://api.spotify.com/v1/tracks/2EBCVPNAG46n...,https://api.spotify.com/v1/audio-analysis/2EBC...,210627,4
2,0.647,0.933,7,-4.056,1,0.1110,0.000351,0.002770,0.3340,0.3320,119.921,audio_features,6pGUGTIaZ1H4jKHIL4Fged,spotify:track:6pGUGTIaZ1H4jKHIL4Fged,https://api.spotify.com/v1/tracks/6pGUGTIaZ1H4...,https://api.spotify.com/v1/audio-analysis/6pGU...,235107,4
3,0.733,0.818,10,-7.222,0,0.0859,0.024100,0.000000,0.0636,0.2530,116.019,audio_features,09TcIuH1ZO7i4vicWKoaN2,spotify:track:09TcIuH1ZO7i4vicWKoaN2,https://api.spotify.com/v1/tracks/09TcIuH1ZO7i...,https://api.spotify.com/v1/audio-analysis/09Tc...,232147,4
4,0.692,0.711,0,-7.498,0,0.0317,0.225000,0.000000,0.1200,0.8750,125.135,audio_features,1TfqLAPs4K3s2rJMoCokcS,spotify:track:1TfqLAPs4K3s2rJMoCokcS,https://api.spotify.com/v1/tracks/1TfqLAPs4K3s...,https://api.spotify.com/v1/audio-analysis/1Tfq...,216933,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.545,0.953,7,-5.172,1,0.0441,0.000941,0.117000,0.2810,0.5580,140.002,audio_features,6VfNTf0N1HwfFKl7Y18diU,spotify:track:6VfNTf0N1HwfFKl7Y18diU,https://api.spotify.com/v1/tracks/6VfNTf0N1Hwf...,https://api.spotify.com/v1/audio-analysis/6VfN...,216027,4
9996,0.337,0.977,9,-1.995,1,0.0976,0.004920,0.872000,0.2600,0.0582,169.867,audio_features,28d1X9lfagOD4iFULH4qEK,spotify:track:28d1X9lfagOD4iFULH4qEK,https://api.spotify.com/v1/tracks/28d1X9lfagOD...,https://api.spotify.com/v1/audio-analysis/28d1...,345706,4
9997,0.726,0.903,7,-2.277,0,0.0357,0.055000,0.000225,0.1580,0.9580,129.986,audio_features,5HiSc2ZCGn8L3cH3qSwzBT,spotify:track:5HiSc2ZCGn8L3cH3qSwzBT,https://api.spotify.com/v1/tracks/5HiSc2ZCGn8L...,https://api.spotify.com/v1/audio-analysis/5HiS...,211244,4
9998,0.645,0.891,6,-2.505,0,0.0387,0.093200,0.000004,0.3790,0.5680,124.915,audio_features,2rgtCOVK2tpfao2VjURn84,spotify:track:2rgtCOVK2tpfao2VjURn84,https://api.spotify.com/v1/tracks/2rgtCOVK2tpf...,https://api.spotify.com/v1/audio-analysis/2rgt...,203520,4


In [14]:
data= pd.merge(df1, df2, left_index=True, right_index=True)
data

,Track_id,Title,Artist_id,Artists,Album,Album Release Date,danceability,energy,key,loudness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,3tgZ9vmhuAY9wEoNUJskzV,Timber (feat. Ke$ha),0TnOYISbd1XYRBk9myaseg,"Pitbull, Kesha",Global Warming: Meltdown (Deluxe Version),2012,0.589,0.966,11,-4.106,...,0.1470,0.8020,129.983,audio_features,3tgZ9vmhuAY9wEoNUJskzV,spotify:track:3tgZ9vmhuAY9wEoNUJskzV,https://api.spotify.com/v1/tracks/3tgZ9vmhuAY9...,https://api.spotify.com/v1/audio-analysis/3tgZ...,204200,4
1,2EBCVPNAG46nbgs6jXPGvv,Starships,0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,Pink Friday: Roman Reloaded The Re-Up (Explici...,2012-01-01,0.747,0.716,11,-2.457,...,0.2510,0.7510,125.008,audio_features,2EBCVPNAG46nbgs6jXPGvv,spotify:track:2EBCVPNAG46nbgs6jXPGvv,https://api.spotify.com/v1/tracks/2EBCVPNAG46n...,https://api.spotify.com/v1/audio-analysis/2EBC...,210627,4
2,6pGUGTIaZ1H4jKHIL4Fged,Wild Strawberries,6n28c9qs9hNGriNa72b26u,PNAU,PNAU (Tour Edition),2009-01-01,0.647,0.933,7,-4.056,...,0.3340,0.3320,119.921,audio_features,6pGUGTIaZ1H4jKHIL4Fged,spotify:track:6pGUGTIaZ1H4jKHIL4Fged,https://api.spotify.com/v1/tracks/6pGUGTIaZ1H4...,https://api.spotify.com/v1/audio-analysis/6pGU...,235107,4
3,09TcIuH1ZO7i4vicWKoaN2,Papaoutai,5j4HeCoUlzhfWtjAfM1acR,Stromae,racine carrée,2013-01-01,0.733,0.818,10,-7.222,...,0.0636,0.2530,116.019,audio_features,09TcIuH1ZO7i4vicWKoaN2,spotify:track:09TcIuH1ZO7i4vicWKoaN2,https://api.spotify.com/v1/tracks/09TcIuH1ZO7i...,https://api.spotify.com/v1/audio-analysis/09Tc...,232147,4
4,1TfqLAPs4K3s2rJMoCokcS,Sweet Dreams (Are Made of This) - Remastered,0NKDgy9j66h3DLnN8qu1bB,"Eurythmics, Annie Lennox, Dave Stewart",Sweet Dreams (Are Made Of This),1983-01-04,0.692,0.711,0,-7.498,...,0.1200,0.8750,125.135,audio_features,1TfqLAPs4K3s2rJMoCokcS,spotify:track:1TfqLAPs4K3s2rJMoCokcS,https://api.spotify.com/v1/tracks/1TfqLAPs4K3s...,https://api.spotify.com/v1/audio-analysis/1Tfq...,216933,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,6VfNTf0N1HwfFKl7Y18diU,Omen,4k1ELeJKT1ISyDv8JivPpB,The Prodigy,Invaders Must Die,2009-02-23,0.545,0.953,7,-5.172,...,0.2810,0.5580,140.002,audio_features,6VfNTf0N1HwfFKl7Y18diU,spotify:track:6VfNTf0N1HwfFKl7Y18diU,https://api.spotify.com/v1/tracks/6VfNTf0N1Hwf...,https://api.spotify.com/v1/audio-analysis/6VfN...,216027,4
9996,28d1X9lfagOD4iFULH4qEK,Dark Star - Homemade Weapons Remix,2rekrvrzMex0PPBX6zvvfj,"Quadrant, Iris, Homemade Weapons",Form Constant EP,2020-04-24,0.337,0.977,9,-1.995,...,0.2600,0.0582,169.867,audio_features,28d1X9lfagOD4iFULH4qEK,spotify:track:28d1X9lfagOD4iFULH4qEK,https://api.spotify.com/v1/tracks/28d1X9lfagOD...,https://api.spotify.com/v1/audio-analysis/28d1...,345706,4
9997,5HiSc2ZCGn8L3cH3qSwzBT,러시안 룰렛 Russian Roulette,1z4g3DjTBBZKhvAroFlhOM,Red Velvet,Russian Roulette - The 3rd Mini Album,2016-09-07,0.726,0.903,7,-2.277,...,0.1580,0.9580,129.986,audio_features,5HiSc2ZCGn8L3cH3qSwzBT,spotify:track:5HiSc2ZCGn8L3cH3qSwzBT,https://api.spotify.com/v1/tracks/5HiSc2ZCGn8L...,https://api.spotify.com/v1/audio-analysis/5HiS...,211244,4
9998,2rgtCOVK2tpfao2VjURn84,Lovers on the Sun (feat. Sam Martin),1Cs0zKBU1kc0i8ypK3B9ai,"David Guetta, Sam Martin",Listen,2014-11-10,0.645,0.891,6,-2.505,...,0.3790,0.5680,124.915,audio_features,2rgtCOVK2tpfao2VjURn84,spotify:track:2rgtCOVK2tpfao2VjURn84,https://api.spotify.com/v1/tracks/2rgtCOVK2tpf...,https://api.spotify.com/v1/audio-analysis/2rgt...,203520,4


In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
data.to_csv("spotify.csv")

In [ ]:
#THIS DIDN'T WORKED


def playlists_data(playlist_id):
    df = []

    for count in range(0, 10000, 100):
        results = sp.user_playlist_tracks(user='spotify', playlist_id=playlist_id, limit=50, offset=count)

        tracks = results['items']
        for item in tracks:
            track_id = item['track']['id']
            title = item['track']['name']
            artist_names = [artist['name'] for artist in item['track']['artists']]
            artists_string = ", ".join(artist_names)
            artist_id = item['track']['artists'][0]['id']
            album = item['track']['album']['name']
            album_release = item['track']['album']['release_date']
            #artist_info = sp.artist(artist_id)
            #genres = artist_info['genres']
            #genre_string = ", ".join(genres)

            audio_features = sp.audio_features(track_id)
            for feature in audio_features:
                danceability = feature['danceability']
                energy = feature['energy']
                key = feature['key']
                loudness = feature['loudness']
                mode = feature['mode']
                speechiness = feature['speechiness']
                acousticness = feature['acousticness']
                instrumentalness = feature['instrumentalness']
                liveness = feature['liveness']
                valence = feature['valence']
                tempo = feature['tempo']

                df.append({
                    'Track_id': track_id,
                    'Title': title,
                    'Artist_id': artist_id,
                    'Artists': artists_string,
                    'Album': album,
                    'Album Release Date': album_release,
                    #'Genre': genre_string,
                    'Danceability': danceability,
                    'Energy': energy,
                    'Key': key,
                    'Loudness': loudness,
                    'Mode': mode,
                    'Speechiness': speechiness,
                    'Acousticness': acousticness,
                    'Instrumentalness': instrumentalness,
                    'Liveness': liveness,
                    'Valence': valence,
                    'Bpms': tempo
                })

    return pd.DataFrame(df)

df1 = playlists_data('6FKDzNYZ8IW1pvYVF4zUN2')
df1